## Q1. Chi Square Test

In [1]:
#TODO


import pandas as pd
import json
import numpy as np

business_path = "/Users/Andy/Desktop/yelp_academic_dataset_business.json"
business_doc=open(business_path, encoding="utf8").readlines()

busList = []
for bus in business_doc:
    busList.append(json.loads(bus))

dfBus = pd.DataFrame.from_records(busList)
dfBus = dfBus.dropna()
from pandas.io.json import json_normalize

In [2]:
open_df = dfBus.loc[:,'is_open']
estrella_df = dfBus.loc[:,'stars']

stars = estrella_df.values

bin_stars = [1 if (star > 2.5) else 0 for star in stars]

bin_stars = np.array(bin_stars) #1 for above 2.5 stars 0 for 2.5 and below

is_open = open_df.values #1 for open 0 for closed


##########################################################################
##########Data gathered, lets make observed & expected dataframes#########
##########################################################################

dfobserved =  pd.DataFrame(columns = ['Business Open' , 'Business Closed', 'Total Stars'], index=['Stars above 2.5', 'Stars below 2.5', 'Total Operating'])

##4 variables for each subcat
open_above = 0
open_below = 0
closed_above = 0
closed_below = 0

for idx, bus_status in enumerate(is_open):

    star_status = bin_stars[idx]

    if bus_status == 1:
        if star_status == 1:
            open_above += 1
        else:
            open_below += 1
    else:
        if star_status == 1:
            closed_above += 1
        else:
            closed_below += 1




In [3]:
#####lets fill in observed df########
dfobserved["Business Open"]["Stars above 2.5"]=open_above
dfobserved["Business Open"]["Stars below 2.5"]=open_below
dfobserved["Business Closed"]["Stars above 2.5"]=closed_above
dfobserved["Business Closed"]["Stars below 2.5"]=closed_below


#totals


dfobserved["Business Open"]["Total Operating"]=dfobserved[['Business Open']].dropna().sum()[0]
dfobserved["Business Closed"]["Total Operating"]=dfobserved[['Business Closed']].dropna().sum()[0]
dfobserved["Total Stars"]["Stars above 2.5"]=dfobserved.loc[["Stars above 2.5"]].T.dropna().sum()[0]
dfobserved["Total Stars"]["Stars below 2.5"]=dfobserved.loc[["Stars below 2.5"]].T.dropna().sum()[0]

dfobserved["Total Stars"]["Total Operating"]=dfobserved[["Total Stars"]].dropna().sum()[0]




In [4]:
import scipy.stats

dfNoTot=dfobserved.loc[["Stars above 2.5","Stars below 2.5"],['Business Open','Business Closed']]

contingency=pd.crosstab(bin_stars,is_open)
contingency.index=["Stars below 2.5","Stars above 2.5"]
contingency.columns=["Business Closed","Business Open"]

##contingecy and dfnoTot should be the same with some column//row swapping

chistat, pval, dof, expected = scipy.stats.chi2_contingency(contingency)


#TODO

conclusions = "The p value is %.3f%% for a 95%% confidence level, this is too low since it is below 5%%\n We reject the null hypothesis. A business's open/closed status and stars on yelp are not independent."%(pval*100)
print('Conclusions:\t', conclusions)

Conclusions:	 The p value is 2.605% for a 95% confidence level, this is too low since it is below 5%
 We reject the null hypothesis. A business's open/closed status and stars on yelp are not independent.


## Q2. Correlation coefficients

In [15]:
user_path = "/Users/Andy/Desktop/yelp_dataset/hw3/yelp_academic_dataset_user_no_friend.json"
user_doc=open(user_path, encoding="utf8").readlines()


userList = []
for user in user_doc:
    userList.append(json.loads(user))

dfUser = pd.DataFrame.from_records(userList)
dfUser = dfUser.dropna()


In [17]:
##generate arrays
num_reviews = dfUser["review_count"]
num_fans = dfUser["fans"]

num_useful=dfUser["useful"]
num_funny=dfUser["funny"]


In [23]:
from scipy.stats import pearsonr

#TODO pearson coefficient
pear_review_fan, _ = pearsonr(num_reviews, num_fans)
pear_useful_funny, _ = pearsonr(num_useful,num_funny)
pear_useful_fan, _ = pearsonr(num_useful,num_fans)


print("Pearson Coefficient Between the number of reviews and the number of fans:%.3f \n Pearson Coefficient Between the number of useful and the number of funny received:%.3f \n Pearson Coefficient Between the number of useful received and the number of fans:%.3f"%(pear_review_fan,pear_useful_fan,pear_useful_funny))

Pearson Coefficient Between the number of reviews and the number of fans:0.538 
 Pearson Coefficient Between the number of useful and the number of funny received:0.681 
 Pearson Coefficient Between the number of useful received and the number of fans:0.964


In [24]:
from scipy.stats import spearmanr
#TODO spearman coefficient
spe_review_fan, _ = spearmanr(num_reviews, num_fans)
spe_useful_funny, _ = spearmanr(num_useful,num_funny)
spe_useful_fan, _ = spearmanr(num_useful,num_fans)


print("Spearman Coefficient Between the number of reviews and the number of fans:%.3f \n Spearman Coefficient Between the number of useful and the number of funny received:%.3f \n Spearman Coefficient Between the number of useful received and the number of fans:%.3f"%(spe_review_fan,spe_useful_fan,spe_useful_funny))


Spearman Coefficient Between the number of reviews and the number of fans:0.535 
 Spearman Coefficient Between the number of useful and the number of funny received:0.549 
 Spearman Coefficient Between the number of useful received and the number of fans:0.774


In [25]:
#TODO
conclusions = 'By just looking at the Pearson coefficient, I would argue reviews and fans / useful and funny have moderate, positive correlation. I would also say useful and fans have very high, positive correlation\n Looking at the Spearman coefficent though, we see useful and fans actually have a high correlation. Not much else changes, all correlations are still positive and the other two pairs don\'t move much. This might indicate useful has a lot of outliers since calculations where useful was considered were moved the most.'
print('Conclusions:\t', conclusions)

Conclusions:	 By just looking at the Pearson coefficient, I would argue reviews and fans / useful and funny have moderate, positive correlation. I would also say useful and fans have very high, positive correlation
 Looking at the Spearman coefficent though, we see useful and fans actually have a high correlation. Not much else changes, all correlations are still positive and the other two pairs don't move much. This might indicate useful has a lot of outliers since calculations where useful was considered were moved the most.


## Q3. Association rule

In [9]:
import re
df_AZ = dfBus[dfBus["state"]=="AZ"]
raw_categories = df_AZ.loc[:,'categories']

cats = []
for cat in raw_categories:
    cats.append(cat)


#TODO


In [10]:
from apyori import apriori
nested = []

for desc in cats:
    sub = []
    key_words = re.findall('[a-zA-Z]+', desc)
    for word in key_words:
        sub.append(word)
    nested.append(sub)

rules = apriori(nested, min_support=0.055, min_confidence=0.85,min_lift=3.75,max_length=2)
rules = filter(lambda x: len(x.items) > 1, rules)
list(rules)

[RelationRecord(items=frozenset({'Life', 'Active'}), support=0.05794847521806748, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Active'}), items_add=frozenset({'Life'}), confidence=1.0, lift=16.487179487179485), OrderedStatistic(items_base=frozenset({'Life'}), items_add=frozenset({'Active'}), confidence=0.955406911928651, lift=16.487179487179485)]),
 RelationRecord(items=frozenset({'Restaurants', 'American'}), support=0.05578470484819798, ordered_statistics=[OrderedStatistic(items_base=frozenset({'American'}), items_add=frozenset({'Restaurants'}), confidence=1.0, lift=4.517564402810304)]),
 RelationRecord(items=frozenset({'Automotive', 'Auto'}), support=0.057249757702797124, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Auto'}), items_add=frozenset({'Automotive'}), confidence=0.9818322381136452, lift=11.42132955122918)]),
 RelationRecord(items=frozenset({'Hair', 'Beauty'}), support=0.06371852953771948, ordered_statistics=[OrderedStatistic(items_base=frozense

In [11]:
#TODO
conclusions = 'I find the Salons and Spas rule interesting. Since the two have a similar function but different experiences \n This rule has a lift of 8.232. So if we randomly select a business tagged as Salon, it is more likely to be tagged as a Spa as well.'
print('Conclusions:\t', conclusions)

Conclusions:	 I find the Salons and Spas rule interesting. Since the two have a similar function but different experiences 
 This rule has a lift of 8.232. So if we randomly select a business tagged as Salon, it is more likely to be tagged as a Spa as well.


## Q4. Power law distribution

In [26]:
review_count = dfBus["review_count"]
#TODO ccdf


In [13]:
#TODO powerlaw


In [14]:
#TODO
conclusions = 'your conclusions'
print('Conclusions:\t', conclusions)


Conclusions:	 your conclusions
